In [3]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [1]:
from parser import get_lyrics_for_song 
from database import Song

In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('data/yandex_tracks_top100.csv')
df.rename(columns={'artist(s)': 'artist', 'name': 'track'}, inplace=True)
df.head()

,track,track_len,link,genre,artist,chart,Explicit_content,monthly_listens_total,artists_likes_total
0,Царица,3:35,https://music.yandex.ru//album/26605629/track/...,русская поп-музыка,['ANNA ASTI'],1,0,4847753,2120528
1,Плохая Девочка,2:15,https://music.yandex.ru//album/26697445/track/...,русский рэп,"['Винтаж', 'ТРАВМА', 'SKIDRI', 'DVRKLXGHT']",2,0,16677658,417013
2,Я плачу,1:54,https://music.yandex.ru//album/26607707/track/...,русская поп-музыка,['A.V.G'],3,0,4406023,287295
3,ASPHALT 8,2:15,https://music.yandex.ru//album/25361122/track/...,русский рэп,['MACAN'],4,0,5149399,2359899
4,Где прошла ты,2:48,https://music.yandex.ru//album/25163882/track/...,русский рэп,"['Кравц', 'Гио Пика']",5,0,9058111,1509523


In [25]:
x = 3

In [29]:
eval('x + 15.0001')

18.0001

In [5]:
df['artist(s)'].apply(lambda x: len(eval(x))).value_counts()

artist(s)
1    71
2    20
3     4
4     3
0     1
5     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [5]:
import requests
import json
from bs4 import BeautifulSoup
from unidecode import unidecode
from tqdm import tqdm

In [34]:
unidecode('ASPHALT 8') 

'ASPHALT 8'

<IPython.core.display.Javascript object>

In [38]:
def get_lyrics_for_song(artist: str, track: str, BASE_URL_LYRICS: str = "https://www.musixmatch.com/lyrics/{}/{}") -> list[str] | AttributeError:
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
    response = requests.get(BASE_URL_LYRICS.format('-'.join(artist.split()), '-'.join(track.split())), headers=headers)
    if response.status_code != 200:
        print(f'haha {response.status_code}')
        return ''

    soup = BeautifulSoup(response.content, "html.parser")
    print(soup.find_all('class="css-175oi2r r-13awgt0"'))
    soup = soup.find("div", {"css-175oi2r r-zd98yo": "true"})
    print(soup)
    lyrics = []
    for elem in soup:
        if line := elem.next_element.text.strip():
            lyrics.append(line)
    print(lyrics[0])
    try:
        return '\n'.join(lyrics)
    except AttributeError as err:
        return ''




In [6]:
API_KEY = '5e8771d5b88f246c0ca497d672868d0b'
URL_TRACK_ID = "https://api.musixmatch.com/ws/1.1/track.search"
URL_TRACK_LYRICS = "https://api.musixmatch.com/ws/1.1/track.lyrics.get"

In [8]:
get_lyrics_from_api('ANNA-ASTI', 'Царица')

161078084


['Все твои романы — тяжёлый вид спорта',
 'Каждый бывший выводил из зоны комфорта',
 'Каждый бывший — тренер личностного роста',
 'Ты стала хитрее, детка стала взрослой',
 '',
 'И на этой дискотеке ты не плачешь под утро',
 'Под любимые треки, маскируя тушь пудрой',
 'Своё сердце обточила в острые, как нож, грани',
 'И оно превратилось в драгоценный, но камень',
 '',
 'Теперь он пьяный по твоей вине',
 'Царица, царица',
 'Один лишь взгляд, и лютый холод по спине',
 'Он просто не может в тебе не раствориться',
 '',
 'Мальчик поплыл, мальчик попал',
 'А как он стесняется, а как он целуется',
 'Мальчик поплыл прям в центре зала',
 'Пусть он танцует, пока танцуется',
 '',
 '...',
 '',
 '******* This Lyrics is NOT for Commercial use *******',
 '(1409625050161)']

In [13]:
def get_lyrics_from_api(artist: str, track: str):
    params_track = {"q_track": track, 'q_artist': artist, 'apikey': API_KEY}
    response = requests.get(URL_TRACK_ID, params=params_track).json()
    try:
        track_id = response['message']['body']['track_list'][0]['track']['commontrack_id']
        params_lyrics = {'commontrack_id': track_id, 'apikey': API_KEY}
        response_lyrics = requests.get(URL_TRACK_LYRICS, params=params_lyrics)
        if response_lyrics.status_code != 200:
            print('wtf')
            return ''
        response_lyrics = response_lyrics.json()
        return response_lyrics['message']['body']['lyrics']['lyrics_body']
    except:
        return ''

In [ ]:
for index, line in tqdm(df.iterrows()):
    df.loc[index, 'lyrics'] = get_lyrics_for_song(eval(line.artist)[0], line.track) if len(eval(line.artist)) == 1 else ''
    
    

In [14]:
for index, line in tqdm(df[15:].iterrows()):
    df.loc[index, 'lyrics'] = get_lyrics_from_api(eval(line.artist)[0], line.track) if len(eval(line.artist)) == 1 else ''
    

85it [01:21,  1.05it/s]


In [ ]:
!pip3 install natasha

In [49]:
import pandas as pd
import numpy as np
from natasha import (
    Segmenter, #tokenization
    MorphVocab, #какя часть речи род и тд

    NewsEmbedding,
    NewsMorphTagger, #
    NewsSyntaxParser,
    NewsNERTagger,

    PER,
    NamesExtractor,

    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

def prepare_words(words: str):
    doc = Doc(words)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    return [token.text.lower() for token in doc.tokens if token.pos != 'PUNCT']
    

lyrics = df.lyrics.dropna().str.replace('******* This Lyrics is NOT for Commercial use *******', '').apply(prepare_words).reset_index(drop=True)#for dop index not emerge

lyrics


0     [все, твои, романы, тяжёлый, вид, спорта, кажд...
1                                                    []
2     [ой, она, топ, во, всех, традициях, у, её, цел...
3     [подо, мной, m, 5, asphalt, 8, воу, у, неё, bi...
4                                                    []
                            ...                        
95    [красивый, мой, изъян, каждый, день, с, тобой,...
96    [у-у-у, эй-эй-эй-эй, открыл, глаза, ещё, дышу,...
97    [ау, ау, ау, ау, ау, ау, ау, ау, ау, ау, ау, а...
98                                                   []
99    [звезды, в, лужах, делить, с, тобой, теплый, у...
Name: lyrics, Length: 100, dtype: object

In [50]:
lyrics = lyrics.apply(lambda x: x[:-1] if len(x) > 0 else [])

In [46]:
def ngram(words: list, n: int = 2) -> list[list[str, str]]:
    return [words[i-n:i] for i in range(n, len(words))]

In [56]:
lyrics = lyrics.apply(ngram)

In [72]:
from collections import Counter

def probability_count(words: list[str, str]):
    prob_dict = {}
    for first, second in words:
        if first in prob_dict:
            prob_dict[first].append(second)
        else:
            prob_dict[first] = [second]
            
    for key in prob_dict.keys():
        prob_dict[key] = Counter(prob_dict[key])
        total = sum(prob_dict[key].values())
        prob_dict[key] = {k: v / total for k, v in prob_dict[key].items()}
    
    return prob_dict

# key: {value1: 2, value2: 3}
    

In [76]:
ngram_distribution = probability_count(sum(lyrics.values, []))
ngram_distribution

{'все': {'твои': 0.13043478260869565,
  'тв': 0.043478260869565216,
  'будут': 0.043478260869565216,
  'разговоры': 0.043478260869565216,
  'танцуй': 0.043478260869565216,
  'окна': 0.043478260869565216,
  'я': 0.043478260869565216,
  'звали': 0.043478260869565216,
  'как': 0.043478260869565216,
  'тебе': 0.043478260869565216,
  'гангстеры': 0.043478260869565216,
  'деньги': 0.043478260869565216,
  'силы': 0.043478260869565216,
  'любят': 0.043478260869565216,
  'смотрят': 0.043478260869565216,
  'постоянно': 0.08695652173913043,
  'свои': 0.043478260869565216,
  'мины': 0.043478260869565216,
  'что': 0.043478260869565216,
  'вот': 0.043478260869565216},
 'твои': {'романы': 0.16666666666666666,
  'секреты': 0.16666666666666666,
  'предки': 0.16666666666666666,
  'девы': 0.16666666666666666,
  'блестят': 0.16666666666666666,
  'как': 0.16666666666666666},
 'романы': {'тяжёлый': 1.0},
 'тяжёлый': {'вид': 1.0},
 'вид': {'спорта': 1.0},
 'спорта': {'каждый': 1.0},
 'каждый': {'бывший': 0.2

In [92]:
import random
start_word = random.choice(list(ngram_distribution.keys()))
print(start_word)
for index in range(30):
    start_word = random.choices(list(ngram_distribution[start_word].keys()),
                                weights=list(ngram_distribution[start_word].values()), k=1)[0]
    print(start_word, end=' ')

искал
твоя улыбка мой рассудок лишь не молчу я влюбился в отделе знаешь о нас глядя плакал голливуд все твои девы постарели как пуля-дура задеть твоя мадлен бонд стоп это мой 